In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc

from bo_methods_lib.GP_Validation_Theta import LOO_In_Theta_Analysis
from bo_methods_lib.bo_functions_generic import gen_theta_set, find_train_doc_path, set_ep, clean_1D_arrays

import matplotlib as mpl

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
# DateTime = "2023/2/24/11-11" #If you want to set a specific time
DateTime = None ##For Testing
print("Date and Time Saved: ", DateTime)

#Set Parameters
#Need to run at a and b, need 2 arrays to test that this will work
CS = 2.2

Bound_Cut = True

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])
if CS == 2.2:
    skip_param_types = 1 #This is what changes for subpoint
    true_p = Constants[skip_param_types:skip_param_types+2].flatten()
    param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
                  4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
    exp_d = 2 
    
    if Bound_Cut == True:
        n = 25 #Number of experimental data points to use
        bounds_x = np.array([[-1.0, 0.0],
                     [   0.5,    1.5]])
    else:
        n = 27 #Number of experimental data points to use
        bounds_x = np.array([[-1.5, -0.5],
                     [   1,    2]])
        
    bounds_p = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                   [ 2,  2,   0,  2,  2,  2, 15,  2]])

else:
    Constants = true_p = np.array([1,-1])
    skip_param_types = 0
    param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
    exp_d = 1
    n = 5
    bounds_x = np.array([[-2], [2]])
    bounds_p = np.array([[-2, -2],
                         [ 2,  2]])

# print(Theta_True)
t_list = np.array([40])
d = len(true_p)
train_iter = 300
noise_std = 0.1
sep_fact = np.linspace(1,1,1)
set_lengthscale = None
explore_bias = 1
plot_axis = np.array([1,0])
normalizing = np.array([False])

obj = np.array(["obj"])
# obj = np.array(["LN_obj"])
# obj = np.array(["obj"])

emulator = np.array([False, True])
# emulator = np.array([True])
# emulator =  np.array([False])
save_figure = True
# save_figure = False

#Pull Experimental data from CSV
if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""
    
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+cut_bounds+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]

Xexp = clean_1D_arrays(Xexp)

for t in t_list:
    for norm in normalizing:
        print("Norm:", norm)
        for emul in emulator:
            print("Emulator =", emul)
            if emul == False:
                t_use = int(t)
                obj_use = obj
            else:
                t_use = int(t*n)
                obj_use = np.array(["obj"])
            for obj_func in obj_use:
                print("Objective Function =", obj_func)
                all_data_doc = find_train_doc_path(emul, obj_func, d, t_use, bound_cut = Bound_Cut)
                all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=","))
                LOO_In_Theta_Analysis(all_data, Xexp, Yexp, Constants, true_p, emul, obj_func, CS,  
                             skip_param_types = skip_param_types, noise_std = noise_std, DateTime = DateTime, 
                             save_figure= save_figure, plot_axis = plot_axis, normalize = norm, bounds_p = bounds_p, 
                             bounds_x = bounds_x, CutBounds = Bound_Cut)        

Date and Time:  08-Mar-2023 (10:23:08)
Date and Time Saved:  None
Norm: False
Emulator = False
Objective Function = obj
[[-0.84026869  0.12848871]
 [ 0.37976389  0.01668494]
 [-0.4537056   0.90634649]
 [ 0.13547048  1.28113218]
 [-0.85199408  1.0922064 ]
 [ 0.31362918  0.52011344]
 [-0.32542195  0.68770339]
 [-0.17374767  1.21615426]
 [-0.19867026  0.39709997]
 [-0.72243858  0.76944386]
 [-0.04598104  0.31630568]
 [-0.69908056  0.60103211]
 [-0.99121469  0.83153925]
 [-0.60311552  1.00841273]
 [ 0.43075713  0.29327019]
 [-0.50275987  1.37955458]
 [ 0.01417026  1.14053566]
 [ 0.26036852  1.49533099]
 [-0.25948888  0.15279138]
 [ 0.11504191  0.58304263]
 [-0.558       1.442     ]
 [-0.05        0.467     ]
 [ 0.623       0.028     ]
 [-0.82        0.62      ]
 [ 0.22        0.3       ]]
Test_Figs_GP_Val/CSV_Data_CB/GP_Val_Theta_CB/CS_2.2/GP_Error_Emulator/TP_40/obj/len_scl_varies/SSE_gp_val/iter_space
Test_Figs_GP_Val/Figures/GP_Val_Theta_CB/CS_2.2/GP_Error_Emulator/TP_40/obj/len_scl_var